# 41. MNIST CNN

<p style="text-align: right;">
blackdew7@gmail.com<br>
Your name :
</p>

### 텐서플로 실습시간 그 다섯번째!<br>

#### 선행지식
1. TensorFlow 다루기 기초
2. 모델링을 한다는 것에 대한 이해.
3. Supervised Learning 중 Classification에 대한 기본 지식.
4. CNN에 대한 기본 구조와 개념

#### 실습목표
1. CNN의 구조를 Graph로 그려낼 수 있다.
2. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.

#### 사용데이터.

01. Multinomial Classification : http://yann.lecun.com/exdb/mnist/

## 00. 라이브러리 불러오기

In [ ]:
# 메모리 초기화
%reset

In [ ]:
# MNIST and Convolutional Neural Network
import tensorflow as tf
import random

## 01. 데이터 불러오기 & 전처리

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)

In [ ]:
tf.convert_to_tensor(mnist.train.images).get_shape()

### MNIST training dataset
* 총 55,000장
* 가로, 세로 길이 = 28
* 데이터는 28 * 28 = 784 길이의 배열로 담겨있음

## 02. Prepare Variables for a CNN Graph with TF

### Matrix Convolution Example
![image](https://docs.gimp.org/en/images/filters/examples/convolution-calculate.png)

### Clear Graph

In [ ]:
# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

### 02.1 Place Holders

#### Input Layer

In [ ]:
X = tf.placeholder(tf.float32, [None, 28 * 28], name="X")
X_img = tf.reshape(X, [-1, 28, 28, 1], name="X_img") 

# 2차원 array로 변환: 784 -> 28x28x1 (black/white)

In [ ]:
# Layer Shape 확인
X.shape
X_img.shape

#### Output Layer

In [ ]:
Y = tf.placeholder(tf.float32, [None, 10], name="Y")

In [ ]:
# Layer Shape 확인
Y.shape

### 02.3 실습 - Prepare Variables for a CNN Graph with TF 

In [ ]:
'''
위의 02 코드를 모아서 Prepare Variables 코드를 완성해보자.

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''

X = tf.placeholder(tf.float32, shape=[None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, shape=[None, 10])

## 03. Make a CNN Graph with TF

### 03.1 First Hidden Layer

#### Weight for Conv Layer01

In [ ]:
# for Conv Layer 01 filter - shape=(3, 3, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01), name="W1")

#### Convolution Layer

In [ ]:
# Convolution Layer 01 -> (?, 28, 28, 32)
L1_conv = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME', name="L1_conv")

#### Activation Function: ReLU

In [ ]:
L1_conv = tf.nn.relu(L1_conv, name="L1_relu")

#### Max Pooling Layer

In [ ]:
# Pooling Layer 01 -> (?, 14, 14, 32)
L1_pool = tf.nn.max_pool(L1_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], 
                         padding='SAME', name="L1_poos")

#### 실습

In [ ]:
'''
위의 코드를 모아서 First Hidden Layer 코드를 완성해보자

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''
# input X_img: (28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1_conv = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1_conv = tf.nn.relu(L1_conv)
L1_pool = tf.nn.max_pool(L1_conv, 
                         ksize=[1, 2, 2, 1], 
                         strides=[1, 2, 2, 1], padding='SAME')

# L1_conv: (28, 28, 32)
# L1_pool: (14, 14, 32)

### 03.2 Second Hidden Layer

#### 바로 실습

In [ ]:
'''
위의 First Hidden Layer 코드를 참고하여
아래 Second Hidden Layer 코드를 완성해보자

__ <= 이렇게 된 빈 칸을 채워주시면 됩니다. 
'''

# for Conv Layer 02 filter - shape=(3, 3, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01), name="W2")

# Convolution Layer 02 -> (?, 14, 14, 64)
L2_conv = tf.nn.conv2d(L1_pool, W2, strides=[1, 1, 1, 1], padding='SAME', name="L2_conv")
L2_conv = tf.nn.relu(L2_conv, name="L2_relu")

# Pooling Layer 02 -> (?, 7, 7, 64)
L2_pool = tf.nn.max_pool(L2_conv, ksize=[1, 2, 2, 1], 
                         strides=[1, 2, 2, 1], 
                         padding='SAME', name="L2_pool")

### 03.3 Flatten Layer & Fully Connected Layer

#### Flatten Layer

In [ ]:
# Flatten layer - 레이어 펼치기
L_flat = tf.reshape(L2_pool, [-1, 7 * 7 * 64], name="L_flat")
# tf.flattern 확인

#### weight for FC

In [ ]:
# for Final FC 7x7x64 inputs -> 10 outputs
W_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 10], stddev=0.01), 
                   name="W_fc")

#### bias

In [ ]:
# bias
b = tf.Variable(tf.random_normal([10]), name="bias")

#### Fully connected layer

In [ ]:
# logits = W_fc * L_flat + bias
logits = tf.add(tf.matmul(L_flat, W_fc), b, name='logits')

#### 실습

In [ ]:
'''
위의 코드를 모아서 Fully Connected Layer 코드를 완성해보자

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''

L_flat = tf.reshape(L2_pool, [-1, 7 * 7 * 64])
W_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 10], stddev=0.01))
b_fc = tf.Variable(tf.random_normal([10]))
logits = tf.add(tf.matmul(L_flat, W_fc), b_fc)


### 03.4 Cost & Optimizer

#### Hyper parameters - learning late

In [ ]:
learning_rate = 0.001

#### Cost(Loss) Function - SoftMax

In [ ]:
# Cost(loss) function - softmax
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="cost")

#### Optimizer - Adam

In [ ]:
# Optimizer - Adam
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
'''
위의 코드를 모아서 Cost& Optimizer 코드를 완성해보자

여러분이 직접 스크립트를 짜야 합니다.
'''


### 03.5 실습 - Make a CNN Graph with TF

In [ ]:
'''
위의 03 실습 코드를 모아서 CNN Graph 코드를 완성해보자

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01), name="W1")
L1_conv = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME', name="L1_conv")
L1_conv = tf.nn.relu(L1_conv, name="L1_relu")
L1_pool = tf.nn.max_pool(L1_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='L1_pool')
# Pooling Layer 02 -> (?, __, __, __)


# for Conv Layer 02 filter - shape=(3, 3, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01), name="W2")
# Convolution Layer 02 -> (?, 14, 14, 32)
L2_conv = tf.nn.conv2d(L1_pool, W2, strides=[1, 1, 1, 1], padding='SAME', name="L2_conv")
L2_conv = tf.nn.relu(L2_conv, name="L2_relu")
L2_pool = tf.nn.max_pool(L2_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L2_pool")
# Pooling Layer 02 -> (?, __, __, __)


L_flat = tf.reshape(L2_pool, [-1, 7 * 7 * 64], name="L_flat")
W_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 10], stddev=0.01))
b = tf.Variable(tf.random_normal([10]), name="bias")
logits = tf.add(tf.matmul(L_flat, W_fc), b, name='logits')


learning_rate = 0.001
# Cost(loss) function - softmax
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="cost")
# Optimizer - Adam
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## 04. Traning & Evaluation

### 04.1 initialize

In [ ]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

### 04.2 traning test code - 10개 데이터를 가져와서 5회 학습

In [ ]:
# 500개씩 가져와서 학습
batch_size = 500

# 55000개의 데이터 500개씩 110번 가져와야 전체를 학습하게 된다.
n_of_batches = int(mnist.train.num_examples / batch_size)


In [ ]:
# n_of_batches회 동안 batch_size개씩 가져와서 학습 진행

for i in range(n_of_batches):
    X_batch, Y_batch = mnist.train.next_batch(batch_size)
    sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})

    # 학습 상황 디스플레이
    if ((i + 1) % 2 == 0):
        loss = sess.run(cost, feed_dict={X: X_batch, Y: Y_batch})
        print("training cost: %.3f" % loss)

### 04.3 Evaluation

In [ ]:
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy = sess.run(acc, feed_dict={X: mnist.test.images, Y: mnist.test.labels})

print('Accuracy: %.2f' % (accuracy * 100))

### 04.4 실습 - Traning & Evaluation

In [ ]:
'''
위의 04 코드를 참고해서 
Traning & Evaluation 코드를 완성해보자

__ <= 이렇게 된 빈 칸을 채워주시면 됩니다. 
'''

###########################
# Initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

###########################
# Training

print('Learning started. It takes sometime.')

# 55000개의 데이터 500개씩 110번 가져와야 전체를 학습하게 된다.
batch_size = 500
n_of_batches = int(mnist.train.num_examples / batch_size)

for i in range(n_of_batches):
    X_batch, Y_batch = mnist.train.next_batch(batch_size)
    sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})

    # 학습 상황 디스플레이
    if ((i + 1) % 2 == 0):
        loss = sess.run(cost, feed_dict={X: X_batch, Y: Y_batch})
        print("training cost: %.3f" % loss)

print("Training Complete")

###########################
# Evaluation

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy = sess.run(acc, feed_dict={X: mnist.test.images, Y: mnist.test.labels})

print('Accuracy: %.2f' % (accuracy * 100))

## 05. 실습 해답

### 05.1 Import Library & Prepare MNIST Data

In [ ]:
import tensorflow as tf
from tbij import show_graph # Tensor Board In Jupyter notebook. You can see this later.

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)


### 05.2 Prepare Variables for a CNN Graph with TF

In [ ]:
# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders

X = tf.placeholder(tf.float32, [None, 784], name="X")
X_img = tf.reshape(X, [-1, 28, 28, 1], name="X_img")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


### 05.3 Make a CNN Graph & Run Session with TF

In [ ]:
#########################
# ConvLayer 01 

with tf.name_scope("layer1_conv"):

    # for Conv Layer 01 filter - shape=(3, 3, 1)
    W1 = tf.Variable(tf.random_normal([3, 3, 1, 24], stddev=0.01), name="W1")

    # Convolution Layer 01 -> (?, 28, 28, 32)
    L1_conv = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME', name="L1_conv")
    L1_conv = tf.nn.relu(L1_conv, name="L1_relu")

    # Pooling Layer 01 -> (?, 14, 14, 32)
    L1_pool = tf.nn.max_pool(L1_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L1_max_pool")

    
#########################
# ConvLayer 02

with tf.name_scope("layer2_conv"):

    # for Conv Layer 02 filter - shape=(3, 3, 32)
    W2 = tf.Variable(tf.random_normal([3, 3, 24, 64], stddev=0.01), name="W2")

    # Convolution Layer 02 -> (?, 14, 14, 32)
    L2_conv = tf.nn.conv2d(L1_pool, W2, strides=[1, 1, 1, 1], padding='SAME', name="L2_conv")
    L2_conv = tf.nn.relu(L2_conv, name="L2_relu")

    # Pooling Layer 02 -> (?, 7, 7, 64)
    L2_pool = tf.nn.max_pool(L2_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L2_max_pool")

    
#########################
# Flatten layer & Fully Connected Layer

with tf.name_scope("L_FC"):

    ###############################
    # Flatten layer
    L_flat = tf.reshape(L2_pool, [-1, 7 * 7 * 64], name="L_flat")

    ###############################
    # Fully Connected Layer

    # for Final FC 7x7x64 inputs -> 10 outputs
    W_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 10], stddev=0.01), name="W_fc")

    # bias
    b = tf.Variable(tf.random_normal([10]), name="b")
    
    # fc layer
    logits = tf.add(tf.matmul(L_flat, W_fc), b, name='logits')

    
#########################
# Cost & Optimizer

learning_rate = 0.001

with tf.name_scope("Optimizer"):
    # Cost(loss) function & Optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="cost")
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


### 05.4 Training & Evaluation

In [ ]:
#########################
# Session initialize

sess = tf.Session()
sess.run(tf.global_variables_initializer())


###########################
# Traning

print('Learning started. It takes sometime.')

# 55000개의 데이터 500개씩 110번 가져와야 전체를 학습하게 된다.
batch_size = 500
n_of_batches = int(mnist.train.num_examples / batch_size)

# n_of_batches회 동안 batch_size개씩 가져와서 학습 진행
for i in range(n_of_batches):
    X_batch, Y_batch = mnist.train.next_batch(batch_size)
    sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})

    # 학습 상황 디스플레이
    if ((i + 1) % 2 == 0):
        loss = sess.run(cost, feed_dict={X: X_batch, Y: Y_batch})
        print("training cost: %.3f" % loss)
        
print("Training Complete")


###########################
# Evaluation
with tf.name_scope("Prediction"):
    # Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1), name="compare")
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    accuracy = sess.run(acc, feed_dict={X: mnist.test.images, Y: mnist.test.labels})

print('Accuracy: %.2f' % (accuracy * 100))

#### 그래프도 그려보자

In [ ]:
from tbij import show_graph # Tensor Board In Jupyter notebook. You can see this later.
show_graph(tf.get_default_graph().as_graph_def())

### 성능을 한 번 확인해 보자. 제대로 되고 있는걸까?

In [ ]:
import random

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

import matplotlib.pyplot as plt
plt.imshow(mnist.test.images[r:r + 1].reshape(28, 28), cmap='Greys', interpolation='nearest')
plt.show()

### 05.5 More Training

#### build cnn

In [ ]:
# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders

X = tf.placeholder(tf.float32, [None, 784], name="X")
X_img = tf.reshape(X, [-1, 28, 28, 1], name="X_img")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")


#########################
# ConvLayer 01 

with tf.name_scope("layer1_conv"):

    # for Conv Layer 01 filter - shape=(3, 3, 1)
    W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01), name="W1")

    # Convolution Layer 01 -> (?, 28, 28, 32)
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME', name="L1_conv")
    L1 = tf.nn.relu(L1, name="L1_relu")

    # Pooling Layer 01 -> (?, 14, 14, 32)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L1_max_pool")

    
#########################
# ConvLayer 02

with tf.name_scope("layer2_conv"):

    # for Conv Layer 02 filter - shape=(3, 3, 32)
    W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01), name="W2")

    # Convolution Layer 02 -> (?, 14, 14, 64)
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME', name="L2_conv")
    L2 = tf.nn.relu(L2, name="L2_relu")

    # Pooling Layer 02 -> (?, 7, 7, 64)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L2_max_pool")

    
#########################
# Fully Connected Layer

with tf.name_scope("L_FC"):

    ###############################
    # Flatten layer
    L_flat = tf.reshape(L2, [-1, 7 * 7 * 64], name="L_flat")

    ###############################
    # Fully Connected Layer

    # for Final FC 7x7x64 inputs -> 10 outputs
    W_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 10], stddev=0.01), name="W_fc")

    # bias
    b = tf.Variable(tf.random_normal([10]), name="b")
    
    # fc layer
    logits = tf.add(tf.matmul(L_flat, W_fc), b, name='logits')

    
#########################
# Cost & Optimizer

learning_rate = 0.001

with tf.name_scope("Optimizer"):
    # Cost(loss) function & Optimizer
    cost = tf.reduce_mean(
             tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="cost")
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


#### traning epoch & evaluation

In [ ]:
#########################
# Session initialize

sess = tf.Session()
sess.run(tf.global_variables_initializer())

###########################
# Traning

# train my model
print('Learning started. It takes sometime.')
        
epochs = 2
batch_size = 500

for epoch in range(epochs):
    print("%dth epoch" % epoch)

    # 총 갯수: 55000개를 500개씩 (batch_size) 나누어 훈련 
    #        => 110번 (n_of_batches) 훈련하게 된다. 
    n_of_batches = int(mnist.train.num_examples / batch_size)
    
    for i in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})
        
        if ((i + 1) % 2 == 0):
            # 학습 상황 디스플레이
            loss = sess.run(cost, feed_dict={X: X_batch, Y: Y_batch})
            print("%dth records, training cost: %.3f" % (((i + 1) * batch_size), loss))
        
        
print("Training Complete")

###########################
# Evaluation

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1, name="predict"), tf.argmax(Y, 1, name="real"), name="compare")
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
accuracy= sess.run(acc, feed_dict={X: mnist.test.images, Y: mnist.test.labels})

print('Accuracy: %.2f' % (accuracy * 100))


## 06. 대체 뭘 학습한 것인가? 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np

def plot_conv_weights(weights, input_channel=0):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        w = sess.run(weights)

        w_min = np.min(w)
        w_max = np.max(w)

        num_filters = w.shape[3]
        num_grids = math.ceil(math.sqrt(num_filters))
        fig, axes = plt.subplots(num_grids, num_grids)

        for i, ax in enumerate(axes.flat):
            if i < num_filters:
                img = w[:, :, input_channel, i]
                ax.imshow(img, vmin=w_min, vmax=w_max, interpolation='nearest', cmap='seismic')

            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()

In [ ]:
plot_conv_weights(weights=W1)

In [ ]:
plot_conv_weights(weights=W2, input_channel=0)

In [ ]:
plot_conv_weights(weights=W2, input_channel=1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np

def plot_conv_layer(layer, image):

    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())

        feed_dict = {X: [image]}
        values = sess.run(layer, feed_dict=feed_dict)

        num_filters = values.shape[3]
        num_grids = math.ceil(math.sqrt(num_filters))
        fig, axes = plt.subplots(num_grids, num_grids)

        for i, ax in enumerate(axes.flat):
            if i < num_filters:
                img = values[0, :, :, i]
                ax.imshow(img, interpolation='nearest', cmap='binary')
                
            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()

In [ ]:
img_shape = (28, 28)

def plot_image(image):
    plt.imshow(image.reshape(img_shape),
               interpolation='nearest',
               cmap='binary')

    plt.show()
    
image1 = mnist.test.images[0]
plot_image(image1)

image2 = mnist.test.images[1]
plot_image(image2)

In [ ]:
plot_conv_layer(layer=L1_pool, image=image2)

In [ ]:
plot_conv_layer(layer=L2_pool, image=image2)

In [ ]:
session.close()